In [387]:
import keras
import sklearn.model_selection
from sklearn.ensemble import AdaBoostClassifier
from sklearn.utils import shuffle
import tensorflow.compat.v1 as tf
from xgboost import XGBClassifier
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# import seaborn as sns
# import matplotlib.pyplot as plt

tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

defined a file path for a job_data.csv and read it into a pandas DataFrame called df, and printed the first 3 rows of the DataFrame to examine the data.

In [388]:
address = r'jobs_in_data.csv'  # Defining the file path for the CSV file
df = pd.read_csv(address)  # Reading the data from CSV file into a DataFrame 'df'

df.head(3) # Printing the first 3 rows of the DataFrame to examine the data

,work_year,job_title,job_category,salary_currency,salary,salary_in_usd,employee_residence,experience_level,employment_type,work_setting,company_location,company_size
0,2023,Data DevOps Engineer,Data Engineering,EUR,88000,95012,Germany,Mid-level,Full-time,Hybrid,Germany,L
1,2023,Data Architect,Data Architecture and Modeling,USD,186000,186000,United States,Senior,Full-time,In-person,United States,M
2,2023,Data Architect,Data Architecture and Modeling,USD,81800,81800,United States,Senior,Full-time,In-person,United States,M


 Perform one-hot encoding on categorical columns

In [389]:
df_encoded = pd.get_dummies(df)
df_encoded.head() # Printing the first 5 rows of the DataFrame to examine the data

,work_year,salary,salary_in_usd,job_title_AI Architect,job_title_AI Developer,job_title_AI Engineer,job_title_AI Programmer,job_title_AI Research Engineer,job_title_AI Scientist,job_title_AWS Data Architect,...,company_location_Switzerland,company_location_Thailand,company_location_Turkey,company_location_Ukraine,company_location_United Arab Emirates,company_location_United Kingdom,company_location_United States,company_size_L,company_size_M,company_size_S
0,2023,88000,95012,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,2023,186000,186000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
2,2023,81800,81800,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
3,2023,212000,212000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
4,2023,93300,93300,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False


In [390]:
# Splitting the data into features (X) and target variable (y)
X = df_encoded.drop(columns=['salary','salary_in_usd'])  # Drop the target column
y = df_encoded['salary']  # Target column

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating a linear regression model
model = LinearRegression()

# Training the model
model.fit(X_train, y_train)

# Predicting on the testing set
y_pred = model.predict(X_test)

# Calculating the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 2.7860732177114695e+33


The Mean Squared Error after one-hot encoding is 2.7860732177114695e+33.
Now we will try to improve the result.

Assigning development levels to countries based on GDP per capita thresholds and performing one-hot encoding to create additional features.

In [391]:
unique_locations = df['company_location'].unique()

# Printing all unique options in the 'company_location' column
# for location in unique_locations:
#     print(location)

In [392]:
# Placeholder GDP per capita values for each country (in USD)
country_gdp_per_capita = {
    'Germany': 52000,
    'United States': 65000,
    'United Kingdom': 48000,
    'Canada': 45000,
    'Spain': 38000,
    'Ireland': 65000,
    'South Africa': 6500,
    'Poland': 33000,
    'France': 42000,
    'Netherlands': 53000,
    'Luxembourg': 115000,
    'Lithuania': 32000,
    'Portugal': 29000,
    'Gibraltar': 89000,
    'Australia': 54000,
    'Colombia': 14800,
    'Ukraine': 9500,
    'Slovenia': 38000,
    'Romania': 13000,
    'Greece': 29000,
    'India': 2200,
    'Latvia': 32000,
    'Mauritius': 11000,
    'Russia': 11000,
    'Italy': 35000,
    'South Korea': 42000,
    'Estonia': 32000,
    'Czech Republic': 38000,
    'Brazil': 8900,
    'Qatar': 62000,
    'Kenya': 1800,
    'Denmark': 60000,
    'Ghana': 2200,
    'Sweden': 54000,
    'Turkey': 11000,
    'Switzerland': 83000,
    'Andorra': 49000,
    'Ecuador': 6600,
    'Mexico': 9900,
    'Israel': 42000,
    'Nigeria': 2400,
    'Saudi Arabia': 22000,
    'Argentina': 11600,
    'Japan': 42000,
    'Central African Republic': 700,
    'Finland': 53000,
    'Singapore': 65000,
    'Croatia': 27000,
    'Armenia': 4400,
    'Bosnia and Herzegovina': 11900,
    'Pakistan': 1600,
    'Iran': 6000,
    'Bahamas': 31000,
    'Austria': 54000,
    'Puerto Rico': 39100,
    'American Samoa': 14000,
    'Thailand': 7800,
    'Philippines': 3500,
    'Belgium': 51000,
    'Egypt': 3000,
    'Indonesia': 4400,
    'United Arab Emirates': 43000,
    'Malaysia': 13000,
    'Honduras': 2500,
    'Algeria': 4200,
    'Iraq': 5700,
    'China': 11000,
    'New Zealand': 44000,
    'Moldova': 3100,
    'Malta': 45000
}

In [393]:
# Define thresholds for GDP per capita (or any other criterion you choose)
high_income_threshold = 40000  # GDP per capita above this value will be considered first world
middle_income_threshold = 10000  # GDP per capita between this and high_income_threshold will be considered second world

# Create a dictionary to map country names to their corresponding development levels
development_levels = {}
for country in unique_locations:
    # Assign development levels based on GDP per capita
    if country in country_gdp_per_capita:
        if country_gdp_per_capita[country] > high_income_threshold:
            development_levels[country] = 'First World'
        elif country_gdp_per_capita[country] > middle_income_threshold:
            development_levels[country] = 'Second World'
        else:
            development_levels[country] = 'Third World'
    else:
        development_levels[country] = 'Data Not Available'  # Handle missing GDP per capita data

# Create a new column 'development_level' based on the assigned levels
df['development_level'] = df['company_location'].map(development_levels)

# Perform one-hot encoding on the 'development_level' column
df_encoded = pd.get_dummies(df, columns=['development_level'])

# Perform one-hot encoding on all categorical columns
df_encoded = pd.get_dummies(df)

df_encoded.head()

,work_year,salary,salary_in_usd,job_title_AI Architect,job_title_AI Developer,job_title_AI Engineer,job_title_AI Programmer,job_title_AI Research Engineer,job_title_AI Scientist,job_title_AWS Data Architect,...,company_location_Ukraine,company_location_United Arab Emirates,company_location_United Kingdom,company_location_United States,company_size_L,company_size_M,company_size_S,development_level_First World,development_level_Second World,development_level_Third World
0,2023,88000,95012,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
1,2023,186000,186000,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,True,False,False
2,2023,81800,81800,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,True,False,False
3,2023,212000,212000,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,True,False,False
4,2023,93300,93300,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,True,False,False


In [394]:
# Splitting the data into features (X) and target variable (y)
X = df_encoded.drop(columns=['salary','salary_in_usd'])  # Drop the target column
y = df_encoded['salary']  # Target column

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating a linear regression model
model = LinearRegression()

# Training the model
model.fit(X_train, y_train)

# Predicting on the testing set
y_pred = model.predict(X_test)

# Calculating the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 1.044927689722729e+33


The Mean Squared Error after add GDP feature is 1.044927689722729e+33.

Reducing the mean squared error (MSE) from 2.7860732177114695e+33 to 1.044927689722729e+33 indicates an improvement in the performance of our model.

In [395]:
# Interaction Terms
df['work_year_experience'] = df['work_year'] * df['experience_level']
df['work_year_job_category'] = df['work_year'] * df['job_category']
df['salary_divide_dolar'] = df['salary_in_usd']/df['salary']

# Aggregate Statistics
agg_salary_by_job_category = df.groupby('job_category')['salary'].agg(['mean', 'median', 'std']).reset_index()
agg_salary_by_job_category.columns = ['job_category', 'avg_salary_job_category', 'median_salary_job_category', 'std_salary_job_category']
df = df.merge(agg_salary_by_job_category, on='job_category', how='left')

# Temporal Trends (if applicable)
# Example: You can calculate the average salary trend over the years
agg_salary_by_year = df.groupby('work_year')['salary'].mean().reset_index()
agg_salary_by_year.columns = ['work_year', 'avg_salary_year']
df = df.merge(agg_salary_by_year, on='work_year', how='left')

# Perform one-hot encoding on categorical columns
df_encoded = pd.get_dummies(df)

# Splitting the data into features (X) and target variable (y)
X = df_encoded.drop(columns=['salary','salary_in_usd'])  # Drop the target column
y = df_encoded['salary']  # Target column

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating a linear regression model
model = LinearRegression()

# Training the model
model.fit(X_train, y_train)

# Predicting on the testing set
y_pred = model.predict(X_test)

# Calculating the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Improved Mean Squared Error:", mse)

Improved Mean Squared Error: 2609546408.356342


In [396]:
# Create a Ridge regression model
ridge_model = Ridge(alpha=1.0)  # Alpha is the regularization strength

# Fit the model to the training data
ridge_model.fit(X_train, y_train)

# Make predictions
y_pred_ridge = ridge_model.predict(X_test)

In [397]:
# Calculate the mean squared error for Ridge Regression
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
print("Mean Squared Error (Ridge Regression):", mse_ridge)

Mean Squared Error (Ridge Regression): 2602098554.267593


Mean Squared Error (Ridge Regression): 2602098554.267593


In [398]:
# Define and train the Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust hyperparameters such as n_estimators
rf_model.fit(X_train, y_train)

# Predict on the testing set
y_pred_rf = rf_model.predict(X_test)

# Calculate the Mean Squared Error for Random Forest Regressor
mse_rf = mean_squared_error(y_test, y_pred_rf)
print("Mean Squared Error (Random Forest):", mse_rf)

Mean Squared Error (Random Forest): 2598873893.6778493
